# 1. Transfer learning: Shallow learning vs Finetuning, Pytorch cont

## 1.2. Feature Extraction

In [1]:
# Encode output into dummy variables
import torch
from torch import nn
from torchvision.models import resnet50
from torch.utils.data import DataLoader, TensorDataset

# preprocess images via PyTorch docs
from torchvision import transforms
from sklearn.preprocessing import OneHotEncoder

# utils
import os
import numpy as np
from PIL import Image
from json import load

In [2]:
def get_raw_data(ann):
    X_raw = []
    y = []
    for img_data, ann_data in zip(ann['images'], ann['annotations']):
        assert ann_data['id'] == img_data['id']
        y.append(ann_data['category_id'])
        img = Image.open('decathlon-1.0/' + img_data['file_name'])
        X_raw.append(np.array(img))
        img.close()
    return X_raw, y

def preprocess(X_raw, transformer_fn):
    X = []
    for img in X_raw:
        img_pil = Image.fromarray(img)
        proc_img = transformer_fn(img_pil)
        X.append(proc_img)
    return torch.stack(X)

In [3]:
with open('decathlon-1.0/annotations/vgg-flowers_train.json') as f:
    ann_train = load(f)
    
with open('decathlon-1.0/annotations/vgg-flowers_val.json') as f:
    ann_val = load(f)

X_train_raw, y_train = get_raw_data(ann_train)
X_val_raw, y_val = get_raw_data(ann_val)


dummifyer = OneHotEncoder(sparse = False)
dummifyer.fit(np.array(y_train).reshape(-1, 1))

y_train_dummy = torch.from_numpy(dummifyer.transform(np.array(y_train).reshape(-1, 1)))
y_val_dummy = torch.from_numpy(dummifyer.transform(np.array(y_val).reshape(-1, 1)))

In [4]:
# hyperparams

# set the same value of hyperparameters (learning rate=0.001, momentum=0.9) for all the layers
momentum = 0.9
batch_size = 64

loss_fn = torch.nn.CrossEntropyLoss()

In [5]:
# preprocess images
transformer_fn = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

X_train = preprocess(X_train_raw, transformer_fn)
X_val = preprocess(X_val_raw, transformer_fn)

# make data loader
train_loader = DataLoader(TensorDataset(X_train, y_train_dummy), batch_size = batch_size, shuffle = True)
val_loader = DataLoader(TensorDataset(X_val, y_val_dummy), batch_size = batch_size, shuffle = True)

In [6]:
# move data to GPU
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

torch.cuda.empty_cache()

cuda:0


In [7]:
# helper function for generating a fresh ResNet
def get_resnet_feature_extractor(num_classes):
    resnet = resnet50(pretrained = True)
    
    # freeze layers
    for params in resnet.parameters():
        params.requires_grad = False
    
    # change the final fully connected layer output to the number of classes in the target dataset.
    resnet.fc = nn.Linear(2048, num_classes)
    resnet.fc.requires_grad = True # unfreeze top layer
    
    return resnet

In [8]:
lrs = [1, 0.1, 0.01, 0.001]
#lrs = [1]

nets = [get_resnet_feature_extractor(102) for i in range(len(lrs))]

In [9]:
# train loop
def train(model, tloader, vloader, lf, optim, epochs = 10):
    
    train_loss_history = []
    val_loss_history = []
    
    for t in range(epochs):

        epoch_train_loss = 0
        epoch_val_loss = 0
        model.train()
        for i, (X_local, y_local) in enumerate(tloader):
            
            X_local = X_local.to(device)
            y_local = y_local.to(device)

            # Forward pass: compute predicted y by passing x to the model.
            y_pred_local = model(X_local)

            # batch loss
            loss = lf(y_pred_local, y_local)
            epoch_train_loss += loss.item()
            #print("batch: %d/%d\tbatch loss: %.2f" % (i + 1, n_batches, loss.item()))

            # compute gradient
            optim.zero_grad()
            loss.backward()

            # update params
            optim.step()

        # get validation loss
        model.eval()
        for i, (X_local, y_local) in enumerate(vloader):
            
            X_local = X_local.to(device)
            y_local = y_local.to(device)

            # disable autograd
            with torch.no_grad():
                y_pred_local = model(X_local)

                # batch loss
                loss = lf(y_pred_local, y_local)
                epoch_val_loss += loss.item()

        # show epoch validation loss
        train_loss_history.append(epoch_train_loss)
        val_loss_history.append(epoch_val_loss)

        print("\t\t\t\tEpoch: %d/%d\tTrain loss: %.2f\tValid loss: %.2f" 
              % (t + 1, epochs, epoch_train_loss, epoch_val_loss))
    
    return train_loss_history, val_loss_history

In [10]:
def accuracy(model, loader, encoder):

    correct = 0
    n = 0
    
    model.to(device)
    model.eval()
    for X_local, y_local in loader:
        
        X_local = X_local.to(device)
        y_local = y_local.to(device)
        
        with torch.no_grad():
            y_logits = model(X_local)

            y_pred = torch.argmax(y_logits, axis = 1)
            y_true = torch.argmax(y_local, axis = 1)
    
        correct += sum(y_pred == y_true).item()
        n += len(y_true)
    
    model.to('cpu')
    return correct / n

In [11]:
epochs = 200

for resnet, learning_rate in zip(nets, lrs):
    
    print('\n\n' + '=' * 40 + '\nlearning rate %.2e\n' % learning_rate + '=' * 40)
    
    resnet.to(device)
    optimizer = torch.optim.SGD(resnet.parameters(), lr = learning_rate, momentum = momentum)
    
    train_loss, val_loss = train(
        model = resnet, tloader =  train_loader, vloader = val_loader, 
        lf = loss_fn, epochs = epochs, optim = optimizer)
    
    print('final train loss: %.3f\tfinal val loss: %.3f' % (train_loss[-1], val_loss[-1]))
    resnet.to('cpu')



learning rate 1.00e+00
				Epoch: 1/200	Train loss: 1778.09	Valid loss: 4419.54
				Epoch: 2/200	Train loss: 5540.22	Valid loss: 5221.01
				Epoch: 3/200	Train loss: 4157.00	Valid loss: 3441.71
				Epoch: 4/200	Train loss: 2011.60	Valid loss: 1691.89
				Epoch: 5/200	Train loss: 924.23	Valid loss: 1050.89
				Epoch: 6/200	Train loss: 414.79	Valid loss: 611.66
				Epoch: 7/200	Train loss: 118.17	Valid loss: 481.75
				Epoch: 8/200	Train loss: 48.77	Valid loss: 412.24
				Epoch: 9/200	Train loss: 27.03	Valid loss: 396.76
				Epoch: 10/200	Train loss: 17.64	Valid loss: 338.52
				Epoch: 11/200	Train loss: 14.19	Valid loss: 342.59
				Epoch: 12/200	Train loss: 12.55	Valid loss: 344.38
				Epoch: 13/200	Train loss: 12.93	Valid loss: 315.61
				Epoch: 14/200	Train loss: 5.96	Valid loss: 306.62
				Epoch: 15/200	Train loss: 3.59	Valid loss: 348.02
				Epoch: 16/200	Train loss: 2.22	Valid loss: 349.06
				Epoch: 17/200	Train loss: 2.50	Valid loss: 314.52
				Epoch: 18/200	Train loss: 1.93	Va

				Epoch: 150/200	Train loss: 0.00	Valid loss: 280.70
				Epoch: 151/200	Train loss: 0.00	Valid loss: 280.69
				Epoch: 152/200	Train loss: 0.00	Valid loss: 280.45
				Epoch: 153/200	Train loss: 0.00	Valid loss: 278.98
				Epoch: 154/200	Train loss: 0.00	Valid loss: 278.71
				Epoch: 155/200	Train loss: 0.00	Valid loss: 280.30
				Epoch: 156/200	Train loss: 0.00	Valid loss: 280.37
				Epoch: 157/200	Train loss: 0.00	Valid loss: 280.25
				Epoch: 158/200	Train loss: 0.00	Valid loss: 277.81
				Epoch: 159/200	Train loss: 0.00	Valid loss: 280.11
				Epoch: 160/200	Train loss: 0.08	Valid loss: 283.18
				Epoch: 161/200	Train loss: 0.20	Valid loss: 273.69
				Epoch: 162/200	Train loss: 0.00	Valid loss: 277.42
				Epoch: 163/200	Train loss: 0.00	Valid loss: 275.87
				Epoch: 164/200	Train loss: 0.00	Valid loss: 277.80
				Epoch: 165/200	Train loss: 0.00	Valid loss: 278.80
				Epoch: 166/200	Train loss: 0.00	Valid loss: 277.17
				Epoch: 167/200	Train loss: 0.00	Valid loss: 279.73
				Epoch:

				Epoch: 100/200	Train loss: 0.05	Valid loss: 21.41
				Epoch: 101/200	Train loss: 0.05	Valid loss: 21.41
				Epoch: 102/200	Train loss: 0.04	Valid loss: 21.46
				Epoch: 103/200	Train loss: 0.04	Valid loss: 21.44
				Epoch: 104/200	Train loss: 0.04	Valid loss: 21.45
				Epoch: 105/200	Train loss: 0.04	Valid loss: 21.41
				Epoch: 106/200	Train loss: 0.04	Valid loss: 21.39
				Epoch: 107/200	Train loss: 0.04	Valid loss: 21.44
				Epoch: 108/200	Train loss: 0.04	Valid loss: 21.36
				Epoch: 109/200	Train loss: 0.04	Valid loss: 21.43
				Epoch: 110/200	Train loss: 0.04	Valid loss: 21.43
				Epoch: 111/200	Train loss: 0.04	Valid loss: 21.53
				Epoch: 112/200	Train loss: 0.04	Valid loss: 21.56
				Epoch: 113/200	Train loss: 0.04	Valid loss: 21.43
				Epoch: 114/200	Train loss: 0.04	Valid loss: 21.46
				Epoch: 115/200	Train loss: 0.04	Valid loss: 21.42
				Epoch: 116/200	Train loss: 0.04	Valid loss: 21.53
				Epoch: 117/200	Train loss: 0.04	Valid loss: 21.44
				Epoch: 118/200	Train los

				Epoch: 50/200	Train loss: 1.54	Valid loss: 21.35
				Epoch: 51/200	Train loss: 1.56	Valid loss: 21.32
				Epoch: 52/200	Train loss: 1.53	Valid loss: 21.22
				Epoch: 53/200	Train loss: 1.48	Valid loss: 21.23
				Epoch: 54/200	Train loss: 1.42	Valid loss: 21.17
				Epoch: 55/200	Train loss: 1.36	Valid loss: 21.13
				Epoch: 56/200	Train loss: 1.36	Valid loss: 21.20
				Epoch: 57/200	Train loss: 1.34	Valid loss: 21.13
				Epoch: 58/200	Train loss: 1.34	Valid loss: 21.12
				Epoch: 59/200	Train loss: 1.28	Valid loss: 21.08
				Epoch: 60/200	Train loss: 1.29	Valid loss: 21.06
				Epoch: 61/200	Train loss: 1.23	Valid loss: 21.07
				Epoch: 62/200	Train loss: 1.15	Valid loss: 21.07
				Epoch: 63/200	Train loss: 1.19	Valid loss: 21.05
				Epoch: 64/200	Train loss: 1.23	Valid loss: 20.96
				Epoch: 65/200	Train loss: 1.18	Valid loss: 21.00
				Epoch: 66/200	Train loss: 1.18	Valid loss: 20.93
				Epoch: 67/200	Train loss: 1.17	Valid loss: 20.92
				Epoch: 68/200	Train loss: 1.15	Valid loss:

				Epoch: 1/200	Train loss: 74.78	Valid loss: 74.03
				Epoch: 2/200	Train loss: 73.31	Valid loss: 72.73
				Epoch: 3/200	Train loss: 71.69	Valid loss: 71.55
				Epoch: 4/200	Train loss: 70.27	Valid loss: 70.40
				Epoch: 5/200	Train loss: 68.79	Valid loss: 69.27
				Epoch: 6/200	Train loss: 67.38	Valid loss: 68.16
				Epoch: 7/200	Train loss: 65.99	Valid loss: 67.10
				Epoch: 8/200	Train loss: 64.63	Valid loss: 66.06
				Epoch: 9/200	Train loss: 63.25	Valid loss: 65.02
				Epoch: 10/200	Train loss: 61.86	Valid loss: 63.98
				Epoch: 11/200	Train loss: 60.65	Valid loss: 62.96
				Epoch: 12/200	Train loss: 59.23	Valid loss: 62.06
				Epoch: 13/200	Train loss: 58.00	Valid loss: 61.12
				Epoch: 14/200	Train loss: 56.75	Valid loss: 60.19
				Epoch: 15/200	Train loss: 55.57	Valid loss: 59.26
				Epoch: 16/200	Train loss: 54.35	Valid loss: 58.38
				Epoch: 17/200	Train loss: 53.11	Valid loss: 57.45
				Epoch: 18/200	Train loss: 52.06	Valid loss: 56.63
				Epoch: 19/200	Train loss: 51.01	V

				Epoch: 153/200	Train loss: 7.77	Valid loss: 25.78
				Epoch: 154/200	Train loss: 7.53	Valid loss: 25.74
				Epoch: 155/200	Train loss: 7.57	Valid loss: 25.73
				Epoch: 156/200	Train loss: 7.62	Valid loss: 25.68
				Epoch: 157/200	Train loss: 7.53	Valid loss: 25.66
				Epoch: 158/200	Train loss: 7.42	Valid loss: 25.64
				Epoch: 159/200	Train loss: 7.45	Valid loss: 25.59
				Epoch: 160/200	Train loss: 7.29	Valid loss: 25.48
				Epoch: 161/200	Train loss: 7.11	Valid loss: 25.45
				Epoch: 162/200	Train loss: 7.08	Valid loss: 25.48
				Epoch: 163/200	Train loss: 6.98	Valid loss: 25.50
				Epoch: 164/200	Train loss: 7.08	Valid loss: 25.31
				Epoch: 165/200	Train loss: 7.11	Valid loss: 25.32
				Epoch: 166/200	Train loss: 6.83	Valid loss: 25.29
				Epoch: 167/200	Train loss: 6.93	Valid loss: 25.29
				Epoch: 168/200	Train loss: 6.79	Valid loss: 25.23
				Epoch: 169/200	Train loss: 6.76	Valid loss: 25.18
				Epoch: 170/200	Train loss: 6.65	Valid loss: 25.15
				Epoch: 171/200	Train los

In [12]:
# Get accuracies

for resnet, learning_rate in zip(nets, lrs):
    print('\n\n' + '=' * 40 + '\nlearning rate %.2e\n' % learning_rate + '=' * 40)
    print("Training Accuracy:\t%.3f" % accuracy(resnet, train_loader, dummifyer))
    print("Validation Accuracy:\t%.3f" % accuracy(resnet, val_loader, dummifyer))



learning rate 1.00e+00
Training Accuracy:	1.000
Validation Accuracy:	0.620


learning rate 1.00e-01
Training Accuracy:	1.000
Validation Accuracy:	0.684


learning rate 1.00e-02
Training Accuracy:	1.000
Validation Accuracy:	0.681


learning rate 1.00e-03
Training Accuracy:	0.999
Validation Accuracy:	0.676
